In [23]:


def cartesian_product(a,b):
    import itertools

    c = list(itertools.product(a, b))
    return c


def combine_to_list(c):
    def flatten(listoflists):    
        flattened_list = [item for sublist in listoflists for item in sublist]
        return flattened_list
    
    # combine the letters into one item
    newlist = []
    
    for i in range(0,len(c)):
        combination = flatten(c[i])
        newlist.append(combination)
            
    return newlist

def modify_to_absorption(c):
    """
    iterate over each line, and if E occurs at any point
    """
    newlist = []
    
    return newlist


def modify_rules(parent, states):
    import numpy as np
    #append probabilities to each row in the condition table
    condprob=[]
        
    #for each parent state
    for parentstate in states:
        
        #subset all rows starting with parent state i
        subset = [row for row in parent if row[0] == parentstate]

        """# manipulate the list """
        
        #All rows, starting with E, should lead only to E
        #If a sequence has E at any point, every subsequent entry becomes E
        
        new_subset = []
        
        for row in subset:
            
            #make a new row, based on rules
            newrow=[]
            
            #flag-variable
            e_observed = False
            
            #for each step in the sequence
            for idx in range(0,len(row)):
                
                
                # if e is observed in current timestep, set flag to true
                if row[idx] == "E":
                    e_observed = True
                
                # 
                if e_observed == True:
                    value = "E"
                else:
                    value = row[idx]
                
                #append new value, based on above logic
                newrow.append(value)
                
                                
            #append new modified row
            new_subset.append(newrow)
        
        #append to final list
        condprob = condprob + new_subset
    
    return condprob


def generate_condprob(parent, states, mode="max_entropy", n_transitions=5):
    import numpy as np
    #append probabilities to each row in the condition table
    condprob=[]
        
    #for each parent state
    for parentstate in states:
        
        #subset all rows starting with parent state i
        subset = [row for row in parent if row[0] == parentstate]

        """# manipulate the list """
        
        #All rows, starting with E, should lead only to E
        #If a sequence has E at any point, every subsequent entry becomes E
        
        if mode=="max_entropy":
            #get list of probabilities for each state
            vec = np.random.random(len(subset))
        
        if mode=="med_entropy":
            #get n random rows with probability > 0, and 0 for rest of the rows
            vec = np.zeros(len(subset)).tolist()
            
            ids = list(range(0,len(vec)))
            import random
            selected = random.sample(ids, n_transitions)
            
            for i in selected:
                vec[i] = np.round(np.random.random(1)[0],decimals=8)
                
        if mode=="min_entropy":
            #get 1 random row with probability == 1 and 0 for rest of the rows
            vec = np.zeros(len(subset)).tolist()
            
            ids = list(range(0,len(vec)))
            import random
            selected = random.sample(ids, 1)[0]
            
            #set probability to 1
            vec[selected] = 1
            
        
        #normalize it
        vec = np.round(vec/np.sum(vec), decimals=5)
        vec = vec.tolist()
        
        for i in range(0,len(subset)):
            #get the probability
            p = vec[i]
            
            #append it to row i in subset
            subset[i].append(p)
            
        #"""
        #append to final list
        condprob = condprob + subset
    
    return condprob

In [21]:

def GenerateInitialProb(D=["a","b"], p0_type="regular"):
    import numpy as np
    import pandas as pd
    
    if p0_type == "min_entropy":
        # Example P0 is one-hot
    
        P0 = np.zeros(len(D))
        P0[np.random.randint(0,len(D),1)[0]] = 1
        P0 = P0.tolist()

    if p0_type != "min_entropy":
        
        P0 = []
    
        for d in D:
            #Draw from uniform dist
            x_d = np.random.uniform(0,1,1)[0]
            #print(x_d)
            #Append the value to the vector
            P0.append(x_d)
        
        #Add the p(absorbtion)=0 to P0
        #P0.append(0)
        
        #Normalize
        S_sum = np.sum(P0)
        P0 = P0/S_sum
        
        #Make dataframe
        #P_0_df = pd.DataFrame(P0).T
        #P_0_df.columns = D
        
    return P0#P_0_df


In [19]:
D = ["A","B","C"]
mode = "max_entropy"

# Including absorption state
D_abs = D.copy()
D_abs.append("!")

In [22]:

# Generate the model

#generate initial probabilities
probabilities = GenerateInitialProb(D_abs, p0_type=mode)    
P0 = {}

for i in range(0,len(D_abs)):
    P0.update({D_abs[i]:probabilities[i]})



In [24]:
states = D
h0 = P0
h=2
mode="max_entropy"
n_transitions=5

    
######################################
# P1

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

#final steps
g = modify_rules(d, states)
p1_input = generate_condprob(g, states, mode, n_transitions)

######################################
# P2

#for each link
c = cartesian_product(states, states)
d = combine_to_list(c)

e = cartesian_product(c, states)
f = combine_to_list(e)

#final steps
g = modify_rules(f, states)
p2_input = generate_condprob(g, states, mode, n_transitions)

### Regular functions

In [32]:
p0.parameters

[{'A': 0.33, 'B': 0.33, 'C': 0.33}]

In [37]:
h0

{'A': 0.1724660961578992,
 'B': 0.11847967043612723,
 'C': 0.5136797486822632,
 '!': 0.1953744847237104}

In [35]:
p1_input

[['A', 'A', 0.36619],
 ['A', 'B', 0.37605],
 ['A', 'C', 0.25776],
 ['B', 'A', 0.08368],
 ['B', 'B', 0.41803],
 ['B', 'C', 0.49829],
 ['C', 'A', 0.14541],
 ['C', 'B', 0.47314],
 ['C', 'C', 0.38145]]

In [34]:
p2_input

[['A', 'A', 'A', 0.02335],
 ['A', 'A', 'B', 0.00575],
 ['A', 'A', 'C', 0.19983],
 ['A', 'B', 'A', 0.05275],
 ['A', 'B', 'B', 0.03855],
 ['A', 'B', 'C', 0.14],
 ['A', 'C', 'A', 0.20077],
 ['A', 'C', 'B', 0.17964],
 ['A', 'C', 'C', 0.15936],
 ['B', 'A', 'A', 0.19726],
 ['B', 'A', 'B', 0.23587],
 ['B', 'A', 'C', 0.06495],
 ['B', 'B', 'A', 0.0625],
 ['B', 'B', 'B', 0.08834],
 ['B', 'B', 'C', 0.1835],
 ['B', 'C', 'A', 0.00884],
 ['B', 'C', 'B', 0.03818],
 ['B', 'C', 'C', 0.12057],
 ['C', 'A', 'A', 0.05249],
 ['C', 'A', 'B', 0.13286],
 ['C', 'A', 'C', 0.20528],
 ['C', 'B', 'A', 0.21217],
 ['C', 'B', 'B', 0.03139],
 ['C', 'B', 'C', 0.02499],
 ['C', 'C', 'A', 0.17645],
 ['C', 'C', 'B', 0.13696],
 ['C', 'C', 'C', 0.02743]]

In [38]:
p0 = DiscreteDistribution(h0)

p1 = ConditionalProbabilityTable(p1_input, [p0])

p2 = ConditionalProbabilityTable(p2_input, [p1])

HOMC = MarkovChain([p0, p1, p2])

In [39]:
HOMC.distributions

In [42]:
HOMC.sample(2)

['C', 'C', 'A']

# Python functions

In [21]:
from distributions.DiscreteDistribution import DiscreteDistribution
d1 = DiscreteDistribution({'A': 0.25, 'B': 0.75})
d1

In [22]:
from distributions.ConditionalProbabilityTable import ConditionalProbabilityTable

d2 = ConditionalProbabilityTable([['A', 'A', 0.1],
                                      ['A', 'B', 0.9],
                                      ['B', 'A', 0.6],
                                      ['B', 'B', 0.4]], [d1])

d3 = ConditionalProbabilityTable([['A', 'A', 'A', 0.4],
                                      ['A', 'A', 'B', 0.6],
                                      ['A', 'B', 'A', 0.8],
                                      ['A', 'B', 'B', 0.2],
                                      ['B', 'A', 'A', 0.9],
                                      ['B', 'A', 'B', 0.1],
                                      ['B', 'B', 'A', 0.2],
                                      ['B', 'B', 'B', 0.8]], [d1, d2])

d2

In [23]:
from MarkovChain import MarkovChain

In [24]:
model = MarkovChain([d1, d2, d3])
model

In [25]:
#print(model.to_json())

In [26]:
model.distributions

In [27]:
model.summarize(sequences=["A","A"])

In [28]:
model.k

2

In [29]:
model.sample

<bound method MarkovChain.sample of <MarkovChain.MarkovChain object at 0x107042a50>>

In [30]:
model.sample(5)

['B', 'A', 'A', 'B', 'A']